# Absorption in Gold with Nevot-Croce vs slabs

In [1]:
import os
from refl1d.names import *
import numpy as np

In [2]:
sys.path.insert(0, os.path.expanduser('~/git/refl1d_analysis/notebooks'))
import plot_utils

## The absorption of the gold is not realistic but was set to see the effect

In [3]:
def make_gold_experiment(q_min=0.005, q_max=0.1, npoints=200, steps=False, gold_thick=850, gold_rough=35):
    q = np.logspace(np.log10(q_min), np.log10(q_max), npoints)
    dq = 0.02*q
    r = np.zeros_like(q)
    probe = QProbe(q, dq, data=(r, r))

    # Materials ####################################################################
    Si = SLD(name='Si', rho=2.067, irho=0.0)
    d2o = SLD(name='Electrolyte', rho=6.05, irho=0.0)
    SiOx = SLD(name='SiOx', rho=3.025, irho=0.0)
    Cr = SLD(name='Cr', rho=2.8, irho=0.0)
    gold = SLD(name='Au', rho=4.264, irho=0.1)
    SAM = SLD(name='SAM', rho=4.0, irho=0.0)


    # Film definition ##############################################################
    sample = (  d2o(0, 4.963) | SAM(10.0, 1.017) | gold(gold_thick, gold_rough) | Cr(60, 1.6) | SiOx(4.94, 2.0) | Si )
    probe.intensity=Parameter(value=0.95,name='normalization')
    probe.background=Parameter(value=5e-06,name='background')
    sample['Electrolyte'].interface.range(1.0, 5.0)

    ################################################################################

    expt = Experiment(probe=probe, sample=sample, step_interfaces=steps, dz=2)
    return expt

def simple_gold_experiment(q_min=0.005, q_max=0.1, npoints=200, steps=False, gold_thick=850, gold_rough=35):
    q = np.logspace(np.log10(q_min), np.log10(q_max), npoints)
    dq = 0.02*q
    r = np.zeros_like(q)
    probe = QProbe(q, dq, data=(r, r))

    Si = SLD(name='Si', rho=2.067, irho=0.0)
    d2o = SLD(name='d2o', rho=6.05, irho=0.0)
    gold = SLD(name='Au', rho=4.264, irho=0.1)

    sample = (  d2o(0, 4.963) | gold(gold_thick, gold_rough) | Si )
    probe.intensity=Parameter(value=0.95,name='normalization')
    probe.background=Parameter(value=5e-06,name='background')

    expt = Experiment(probe=probe, sample=sample, step_interfaces=steps, dz=2)
    return expt



## Reflectivity near weird roughness threshold

In [4]:
gold_rough = 105.5

expt = make_gold_experiment(steps=False, gold_rough=gold_rough)
r_0 = expt.reflectivity()

expt = make_gold_experiment(steps=True, gold_rough=gold_rough)
r_1 = expt.reflectivity()

expt = make_gold_experiment(steps=True, gold_rough=gold_rough+1)
r_2 = expt.reflectivity()

plot_utils.plot1d([
                   [r_0[0], r_0[1]],
                   [r_1[0], r_1[1]],
                   [r_2[0], r_2[1]]
                  ],
                  data_names=['croce, rho=%s' % gold_rough,
                              'steps, rho=%s' % gold_rough,
                              'steps, rho=%s' % (gold_rough+1),
                             ],
                  y_log=True, x_log=True, x_title='q', y_title='R',)

# Model with single gold layer

In [5]:
gold_rough = 171
gold_thick = 500

expt = simple_gold_experiment(steps=False, gold_thick=gold_thick, gold_rough=gold_rough)
r_0 = expt.reflectivity()

expt = simple_gold_experiment(steps=True, gold_thick=gold_thick, gold_rough=gold_rough)
r_1 = expt.reflectivity()

expt = simple_gold_experiment(steps=True, gold_thick=gold_thick, gold_rough=gold_rough+1)
r_2 = expt.reflectivity()

plot_utils.plot1d([
                   [r_0[0], r_0[1]],
                   [r_1[0], r_1[1]],
                   [r_2[0], r_2[1]],
                  ],
                  data_names=['croce, rho=%s' % gold_rough,
                              'steps, rho=%s' % gold_rough,
                              'steps, rho=%s' % (gold_rough+1),
                             ], y_log=True, x_log=True, x_title='q', y_title='R',)
